In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from glob import glob
import os

In [2]:
#Read in data 112K rows
data = pd.read_csv('Data_Entry_2017.csv')
data.drop('Unnamed: 11',axis=1, inplace=True)
data

,Image Index,Finding Labels,Follow-up #,Patient ID,Patient Age,Patient Gender,View Position,OriginalImage[Width,Height],OriginalImagePixelSpacing[x,y]
0,00000001_000.png,Cardiomegaly,0,1,58,M,PA,2682,2749,0.143,0.143
1,00000001_001.png,Cardiomegaly|Emphysema,1,1,58,M,PA,2894,2729,0.143,0.143
2,00000001_002.png,Cardiomegaly|Effusion,2,1,58,M,PA,2500,2048,0.168,0.168
3,00000002_000.png,No Finding,0,2,81,M,PA,2500,2048,0.171,0.171
4,00000003_000.png,Hernia,0,3,81,F,PA,2582,2991,0.143,0.143
...,...,...,...,...,...,...,...,...,...,...,...
112115,00030801_001.png,Mass|Pneumonia,1,30801,39,M,PA,2048,2500,0.168,0.168
112116,00030802_000.png,No Finding,0,30802,29,M,PA,2048,2500,0.168,0.168
112117,00030803_000.png,No Finding,0,30803,42,F,PA,2048,2500,0.168,0.168
112118,00030804_000.png,No Finding,0,30804,30,F,PA,2048,2500,0.168,0.168


In [3]:
#Add specific imagepath to read images into image generator
images = {os.path.basename(x): x for x in 
                   glob(os.path.join('images*', '*', '*.png'))}
data['path'] = data['Image Index'].map(images.get)

In [4]:
#Drop ages over 120
data = data[data['Patient Age']<120]

In [5]:
# Diagnosis to predict
data['clean_labels'] = data['Finding Labels'].apply(lambda x: x.split('|'))
labels = data['clean_labels'].str.join('|').str.get_dummies()
data = data.join(labels)

/home/slindhult/anaconda3/envs/test_env/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [6]:
#Identifying the potential pathologies
len(data['Finding Labels'].unique())

836

In [7]:
data['Finding Labels'] = data['Finding Labels'].map(lambda x: x.replace('No Finding', ''))

all_labels = labels.columns
clean_labels = []
for i in all_labels:
    j = data[i].sum()
    if j >1000 and i != 'No Finding':
        clean_labels.append(i)
    else:
        print(i)


Hernia
No Finding


In [8]:
data.drop(['Hernia', 'No Finding'], axis=1, inplace=True)

In [9]:
hern_dex = [i for i in data.index if 'Hernia' in data['Finding Labels'].loc[i]]

In [10]:
data = data[data['Finding Labels']!='']
data.drop(index=hern_dex, inplace=True)

In [11]:
data['pathology_vec'] = data.apply(lambda x: [x[clean_labels].values], 1).map(lambda x: x[0])

In [13]:
data.clean_labels

0                              [Cardiomegaly]
1                   [Cardiomegaly, Emphysema]
2                    [Cardiomegaly, Effusion]
12                             [Mass, Nodule]
19                             [Infiltration]
                         ...                 
112097    [Consolidation, Pleural_Thickening]
112100                         [Infiltration]
112106                         [Mass, Nodule]
112108                   [Pleural_Thickening]
112115                      [Mass, Pneumonia]
Name: clean_labels, Length: 51524, dtype: object

In [14]:
data.to_csv('sick.csv')